In [6]:
using Pkg, Logging
Pkg.activate(@__DIR__)
push!(LOAD_PATH, joinpath(@__DIR__, "..", ".."))
push!(LOAD_PATH, joinpath(@__DIR__, "..", "..", "extras"))
Pkg.instantiate()
with_logger(NullLogger()) do
    pkg"precompile"
end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Precompiling project...


In [7]:
using JuMP
using LinearAlgebra
using MINLPTrajOpt
using Rotations
using RigidBodyDynamics

In [8]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [ ]:
using TrigonometricPolynomials
using TrigonometricPolynomials: Var

In [ ]:
sincosmap = SinCosDict()
q = [TrigPoly{Float64}(Var("q$i"), sincosmap) for i = 1 : num_positions(mechanism)]
v = [TrigPoly{Float64}(Var("v$i"), sincosmap) for i = 1 : num_velocities(mechanism)]
state = MechanismState(mechanism, q, v)
setdirty!(state)

In [ ]:
M = mass_matrix(state)
map!(simplify, M.data, M.data)
M

In [ ]:
c = dynamics_bias(state)
map!(simplify ∘ simplify, c, c)
c

In [ ]:
c[1]

Design decisions:

* `Problem` type:
  * stores a `JuMP.Model`
  * does all the setup in the constructor
  * stores variables/useful subexpressions
  * solve method
  * methods for accessing results
  * use `AxisArrays` to store variables
* No separate `Result` type
* Implement `TrigPoly` type, run through `mass_matrix` and `dynamics_bias`.

Is this really needed?